In [1]:
%%capture
%pip install transformers datasets accelerate torch evaluate bert_score rouge_score bitsandbytes

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import evaluate

# Load dataset
train_df = pd.read_csv("/kaggle/input/springer-journal-final/train.csv")
val_df = pd.read_csv("/kaggle/input/springer-journal-final/val.csv")
test_df = pd.read_csv("/kaggle/input/springer-journal-final/test.csv")

# Tokenization
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["abstract"])

# Convert text to sequences
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df["abstract"]), padding='post')
Y_train = np.array([seq[0] if len(seq) > 0 else 0 for seq in tokenizer.texts_to_sequences(train_df["title"])]).reshape(-1, 1)
X_val = pad_sequences(tokenizer.texts_to_sequences(val_df["abstract"]), padding='post')
Y_val = np.array([seq[0] if len(seq) > 0 else 0 for seq in tokenizer.texts_to_sequences(val_df["title"])]).reshape(-1, 1)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df["abstract"]), padding='post')
Y_test = test_df["title"].tolist()

# Load GloVe embeddings
glove_path = "/kaggle/input/glove6b100dtxt/glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build GRU model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False),
    GRU(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=32)

# Predict titles
predictions = model.predict(X_test)
predicted_titles = [tokenizer.index_word.get(np.argmax(pred)) for pred in predictions]

# Evaluate model
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
rouge_scores = rouge.compute(predictions=predicted_titles, references=Y_test)
bert_scores = bertscore.compute(predictions=predicted_titles, references=Y_test, lang="en")

print("ROUGE:", rouge_scores)
print("BERTScore (averaged):")
print("  Precision:", sum(bert_scores["precision"]) / len(bert_scores["precision"]))
print("  Recall:", sum(bert_scores["recall"]) / len(bert_scores["recall"]))
print("  F1:", sum(bert_scores["f1"]) / len(bert_scores["f1"]))

Epoch 1/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 101s 68ms/step - accuracy: 0.1114 - loss: 8.0091 - val_accuracy: 0.1151 - val_loss: 6.9258
Epoch 2/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1143 - loss: 6.8010 - val_accuracy: 0.1151 - val_loss: 6.7763
Epoch 3/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1116 - loss: 6.7447 - val_accuracy: 0.1151 - val_loss: 6.8441
Epoch 4/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1124 - loss: 6.7128 - val_accuracy: 0.1151 - val_loss: 6.8848
Epoch 5/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1113 - loss: 6.6960 - val_accuracy: 0.1151 - val_loss: 6.9098
Epoch 6/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1127 - loss: 6.6961 - val_accuracy: 0.1151 - val_loss: 6.9613
Epoch 7/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1109 - loss: 6.6891 - val_accuracy: 0.1151 - val_loss: 6.9825
Epoch 8/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 97s 68ms/step - accuracy: 0.1109 

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE: {'rouge1': 0.03703837801427366, 'rouge2': 0.0, 'rougeL': 0.0371073346787995, 'rougeLsum': 0.0371462985071591}
BERTScore (averaged):
  Precision: 0.837819782197946
  Recall: 0.7814797676042431
  F1: 0.8085472841594036
